In [1]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from packaging import version
from scipy.io import savemat
import time
import sys

In [2]:
class ForceSpectral:
    """
    Description
    """

    def __init__(self, device = 'cuda', dtype = torch.float32, fr = None, c = 1, p1 = 1, resolution=16):
        self.device = device
        self.dtype = dtype


        fr = 7
        self.ka = 1
        print("ka: ", self.ka)
        self.kb = self.ka * fr
        print("kb: ", self.kb)
        self.kf = (self.kb+self.ka)/2
        print("kf: ", self.kf)

        self.c = c
        self.p1 = p1
        self.resolution = resolution

        frequencies = [np.fft.fftfreq(dim, d=1 / dim) for dim in [self.resolution]*3]
        eps = 1e-15
        k = np.array(np.meshgrid(*frequencies))+eps
        kk = np.linalg.norm(k, axis=0)
        k = torch.tensor(np.array(k), device=self.device, dtype=self.dtype)
        self.kk = torch.tensor(np.array(kk), device=self.device, dtype=self.dtype)
        self.k = k
        self.e1 = [k[1] * (k[0]**2 + k[1]**2)**-0.5,
                   -k[0] * (k[0]**2 + k[1]**2)**-0.5,
                   0]
        self.e2 = [k[0]*k[2] * self.kk**-1 * (k[0]**2 + k[1]**2)**-0.5,
                   k[1]*k[2] * self.kk**-1 * (k[0]**2 + k[1]**2)**-0.5,
                  -(k[0]**2 + k[1]**2)**0.5 * self.kk**-1]

    def __call__(self,):
        Fh = self.spectral_force()
        return torch.stack([(torch.fft.ifftn(Fh[i], dim=tuple(torch.arange(3))).real)  for i in range(3)])


    def spectral_force(self,):
        RealRandom, ImagRandom = self.get_complex_factors(self.resolution)

        Fk = torch.exp(-(self.kk-self.kf)**2/self.c)
        Fk*= self.p1 * Fk.sum()**-1
        Fk = torch.sqrt(Fk*(2*torch.pi*self.kk**2)**-1)
        Fk.ravel()[0] = 0

        A = Fk * torch.complex(RealRandom[0],ImagRandom[0])
        B = Fk * torch.complex(RealRandom[1],ImagRandom[1])
        Fh = torch.stack([A*self.e1[_]+B*self.e2[_] for _ in range(3)])
        return Fh

    def get_complex_factors(self, k):
        phi = torch.rand([k]*3, dtype=self.dtype, device=self.device)*torch.pi
        theta = torch.rand([2]+[k]*3, dtype=self.dtype, device=self.device)*2*torch.pi
        ga = torch.sin(2*phi)
        gb = torch.cos(2*phi)

        RealRandom = [torch.cos(theta[_]) * g for _, g in enumerate([ga,gb])]
        ImagRandom = [torch.sin(theta[_]) * g for _, g in enumerate([ga,gb])]
        return RealRandom, ImagRandom

    def _get_random_factors(self, wavenumbers):
        return torch.rand(wavenumbers, dtype=self.dtype, device=self.device)

In [3]:
class ForcedBGKCollision(lt.Collision):
    def __init__(self, lattice, tau, x, Lc, a, k_start = 1, k_end = 2, nu = None):
        self.lattice = lattice
        self.tau = tau
        self.x = x
        self.a = torch.tensor(a, device=lattice.device, dtype=lattice.dtype)
#         self.a = torch.tensor([a,-2*a,a], device=lattice.device, dtype=lattice.dtype)
        self.k = torch.tensor([1,1,1], device=lattice.device, dtype=lattice.dtype)
        self.Lc = torch.tensor(int(Lc), device=lattice.device, dtype=lattice.dtype)
        self.phi = torch.zeros(6, device=lattice.device, dtype=lattice.dtype)
        self.k_end = k_end
        self.k_start = k_start
        self.pi = torch.tensor(np.pi, device=lattice.device, dtype=lattice.dtype)
        self.phase = None
        self.phase_counter = 0
        self.nu = nu
        self.isforce = True
        self.force = ForceSpectral(p1 = a, fr=7 ,c=1, resolution=self.x[0].shape[0])

    def __call__(self, f):

        rho = self.lattice.rho(f)

        u_eq = 0
        Si = 0

#         F = self._F(f)
#         u_eq = 0.5 * F / rho
        u = self.lattice.u(f) + u_eq

#         feq = self.lattice.equilibrium(rho, u)
#         index = [Ellipsis] + [None] * self.lattice.D
#         emu = self.lattice.e[index] - u
#         eu = self.lattice.einsum("ib,b->i", [self.lattice.e, u])
#         eeu = self.lattice.einsum("ia,i->ia", [self.lattice.e, eu])
#         emu_eeu = emu / (self.lattice.cs ** 2) + eeu / (self.lattice.cs ** 4)
#         emu_eeuF = self.lattice.einsum("ia,a->i", [emu_eeu, F])
#         weemu_eeuF = self.lattice.w[index] * emu_eeuF
#         Si = (1 - 1 / (2 * self.tau)) * weemu_eeuF

        feq = self.lattice.equilibrium(rho, u)
        if self.isforce:
            F = self._F(f)
            du = F/rho
            Si = self.lattice.equilibrium(rho, u + du)-feq
        return f - 1.0 / self.tau * (f - feq) + Si

    def _F(self,f=None):
        F = self.force()
        return F



# def convert_powerforce_to_lu(power_force_pu, flow):
def convert_powerforce_to_lu(power_force_pu, flow):
    print("converted pu to lu")
    return (power_force_pu * (flow.units.characteristic_density_lu / flow.units.characteristic_density_pu) *
            (flow.units.characteristic_velocity_lu / flow.units.characteristic_velocity_pu) ** 2 *
            (flow.units.characteristic_length_lu / flow.units.characteristic_length_pu)**6)

In [4]:
load_f = False
steps = 2000
k_start = 1
k_end = 3.5
torch.manual_seed(0)
lattice = lt.Lattice(lt.D3Q27, device = "cuda")
flow = lt.BasicFlow(resolution=32, reynolds_number=256, mach_number=0.05/lattice.stencil.cs, lattice=lattice)
a_pu = 1e3
flow.units.characteristic_velocity_pu = 1
a_lu = convert_powerforce_to_lu(a_pu, flow)
# a_lu = 0

print(a_lu)
print(flow.units.viscosity_pu)
print(flow.units.viscosity_lu)

collision = ForcedBGKCollision(lattice,
                               tau=flow.units.relaxation_parameter_lu,
                               x=flow.units.convert_length_to_lu(torch.tensor(flow.grid, device=lattice.device, dtype=lattice.dtype)),
                               Lc=flow.units.convert_length_to_lu(2*np.pi),
                               a=a_lu,
                               k_start = k_start,
                               k_end = k_end,
                               nu = flow.units.viscosity_lu)

streaming = lt.StandardStreaming(lattice)
simulation = lt.Simulation(flow=flow, lattice=lattice, collision=collision, streaming=streaming)



energyspectrum = lt.EnergySpectrum(lattice, flow)
energy = lt.IncompressibleKineticEnergy(lattice, flow)

reporter_energy_console = lt.ObservableReporter(energy, interval=500)
reporter_energy = lt.ObservableReporter(energy, interval=200, out=None)
reporter_enstrophy = lt.ObservableReporter(lt.Enstrophy(lattice, flow), interval=200, out=None)
reporter_u_max = lt.ObservableReporter(U_max(lattice, flow), interval=200, out=None)
reporter_u_rms = lt.ObservableReporter(U_rms(lattice, flow), interval=200, out=None)
reporter_eps = lt.ObservableReporter(Eps(lattice, flow), interval=200, out=None)
reporter_spectrum = lt.ObservableReporter(energyspectrum, interval=200, out=None)

simulation.reporters.append(reporter_energy_console)
simulation.reporters.append(reporter_energy)
simulation.reporters.append(reporter_enstrophy)
simulation.reporters.append(reporter_u_max)
simulation.reporters.append(reporter_u_rms)
simulation.reporters.append(reporter_eps)
simulation.reporters.append(reporter_spectrum)
simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=200))

simulation.initialize_pressure()
simulation.initialize_f_neq()

spectrum_init = energyspectrum(simulation.f).cpu()
wavenumbers = (np.arange(len(spectrum_init))+1)/k_start



with torch.no_grad():
#     mlups = simulation.step(num_steps=1)
    mlups = simulation.step(num_steps=steps)
print("Performance in MLUPS:", mlups)

converted pu to lu
43627.534280906875
0.02454369260617026
0.0062499999999999995
ka:  1
kb:  7
kf:  4.0


/tmp/ipykernel_77217/3266824935.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/torch/csrc/utils/tensor_new.cpp:201.)
  x=flow.units.convert_length_to_lu(torch.tensor(flow.grid, device=lattice.device, dtype=lattice.dtype)),


steps     time     IncompressibleKineticEnergy
steps     time     IncompressibleKineticEnergy
steps     time     Enstrophy


/home/mbedru3s/programme/miniconda3/envs/lettuce/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


NameError: name 'U_max' is not defined